In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
import csv
from datetime import datetime

# Payoff Matrix and play_pd_round (unchanged)
PAYOFFS = {
    (0, 0): (75, 45),
    (0, 1): (40, 60),
    (1, 0): (90, 30),
    (1, 1): (50, 25)
}

def play_pd_round(action_a, action_b):
    return PAYOFFS.get((action_a, action_b), (0, 0))

# simulate_game (unchanged from last, with mode)
def simulate_game(num_rounds=10, strategy_a='TFT', strategy_b='TFT', noise=0.0, mode='Batch'):
    history_a = []
    history_b = []
    total_payoff_a = 0
    total_payoff_b = 0
    current_payoff_a = 0
    current_payoff_b = 0
    consec_def_b = 0
    consec_def_a = 0
    results = []
    cum_a = []
    cum_b = []

    for round_num in range(num_rounds):
        # Player A's action (logic as before)
        if strategy_a == 'TFT':
            action_a = history_b[-1] if history_b else 0
        elif strategy_a == 'AlwaysCooperate':
            action_a = 0
        elif strategy_a == 'AlwaysDefect':
            action_a = 1
        elif strategy_a == 'Random':
            action_a = np.random.choice([0, 1])
        elif strategy_a == 'GrimTrigger':
            action_a = 1 if 1 in history_b else 0
        elif strategy_a == 'ForgivingTFT':
            if history_b:
                if history_b[-1] == 1:
                    consec_def_b += 1
                else:
                    consec_def_b = 0
                action_a = 1 if consec_def_b >= 2 else 0
            else:
                action_a = 0
        elif strategy_a == 'CaseSpecific':
            avg_profit_a = current_payoff_a / (round_num + 1) if round_num > 0 else 0
            action_a = 1 if avg_profit_a < 50 else 0
        else:
            raise ValueError("Invalid strategy for A")

        # Player B's action (similar)
        if strategy_b == 'TFT':
            action_b = history_a[-1] if history_a else 0
        elif strategy_b == 'AlwaysCooperate':
            action_b = 0
        elif strategy_b == 'AlwaysDefect':
            action_b = 1
        elif strategy_b == 'Random':
            action_b = np.random.choice([0, 1])
        elif strategy_b == 'GrimTrigger':
            action_b = 1 if 1 in history_a else 0
        elif strategy_b == 'ForgivingTFT':
            if history_a:
                if history_a[-1] == 1:
                    consec_def_a += 1
                else:
                    consec_def_a = 0
                action_b = 1 if consec_def_a >= 2 else 0
            else:
                action_b = 0
        elif strategy_b == 'CaseSpecific':
            avg_profit_b = current_payoff_b / (round_num + 1) if round_num > 0 else 0
            action_b = 1 if avg_profit_b < 50 else 0
        else:
            raise ValueError("Invalid strategy for B")

        # Interactive override
        if mode == 'Interactive':
            action_a = int(input(f"Round {round_num + 1} - A action (0/1): "))
            action_b = int(input(f"Round {round_num + 1} - B action (0/1): "))

        # Noise (batch only)
        if mode == 'Batch' and np.random.rand() < noise:
            action_a = 1 - action_a
        if mode == 'Batch' and np.random.rand() < noise:
            action_b = 1 - action_b

        payoff_a, payoff_b = play_pd_round(action_a, action_b)
        history_a.append(action_a)
        history_b.append(action_b)
        total_payoff_a += payoff_a
        total_payoff_b += payoff_b
        current_payoff_a += payoff_a
        current_payoff_b += payoff_b
        results.append((round_num + 1, action_a, action_b, payoff_a, payoff_b))
        cum_a.append(total_payoff_a)
        cum_b.append(total_payoff_b)

        if mode == 'Interactive':
            print(f"Round {round_num + 1}: A={action_a}, B={action_b}, Payoffs A={payoff_a}, B={payoff_b}")

    # Plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(1, num_rounds + 1)), y=cum_a, mode='lines+markers', name='A', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=list(range(1, num_rounds + 1)), y=cum_b, mode='lines+markers', name='B', line=dict(color='orange')))
    fig.update_layout(title=f'PD ({mode}): A={strategy_a}, B={strategy_b}', xaxis_title='Rounds', yaxis_title='Profit (M Rupees)', template='plotly_dark')
    fig.show()

    return results, total_payoff_a, total_payoff_b

# UI with Save
mode_dropdown = widgets.Dropdown(options=['Batch', 'Interactive'], value='Batch', description='Mode:')
strategy_a_dropdown = widgets.Dropdown(options=['TFT', 'AlwaysCooperate', 'AlwaysDefect', 'Random', 'GrimTrigger', 'ForgivingTFT', 'CaseSpecific'], value='TFT', description='A:')
strategy_b_dropdown = widgets.Dropdown(options=['TFT', 'AlwaysCooperate', 'AlwaysDefect', 'Random', 'GrimTrigger', 'ForgivingTFT', 'CaseSpecific'], value='TFT', description='B:')
num_rounds_slider = widgets.IntSlider(min=1, max=50, value=10, description='Rounds:')
noise_slider = widgets.FloatSlider(min=0.0, max=0.5, value=0.0, description='Noise (Batch):')
run_button = widgets.Button(description='Run')
save_button = widgets.Button(description='Save Results', disabled=True)
output = widgets.Output()
latest_results = None  # To store for saving

def on_run_clicked(b):
    global latest_results
    with output:
        clear_output(wait=True)
        results, total_a, total_b = simulate_game(
            num_rounds=num_rounds_slider.value,
            strategy_a=strategy_a_dropdown.value,
            strategy_b=strategy_b_dropdown.value,
            noise=noise_slider.value,
            mode=mode_dropdown.value
        )
        if mode_dropdown.value == 'Batch':
            print("Results:")
            for r in results:
                print(f"Round {r[0]}: A={r[1]}, B={r[2]}, Payoffs A={r[3]}, B={r[4]}")
        print(f"Totals: A={total_a}M, B={total_b}M")
        latest_results = results + [('Totals', '-', '-', total_a, total_b)]  # For save
        save_button.disabled = False

def on_save_clicked(b):
    with output:
        if latest_results:
            filename = f"PD_Run_{strategy_a_dropdown.value}_vs_{strategy_b_dropdown.value}_{datetime.now().strftime('%Y-%m-%d')}.csv"
            with open(filename, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['Round', 'A Action', 'B Action', 'A Payoff', 'B Payoff'])
                writer.writerows(latest_results)
            print(f"Saved to {filename} - Download from Hub files.")

run_button.on_click(on_run_clicked)
save_button.on_click(on_save_clicked)

display(widgets.VBox([mode_dropdown, strategy_a_dropdown, strategy_b_dropdown, num_rounds_slider, noise_slider, run_button, save_button, output]))